# Goodhart's Law: Experiment Runner

This notebook runs training experiments for the Goodhart's Law project.
Works on **Google Colab**, **VS Code Dev Containers**, and **Local Environments**.

## 1. Environment Setup

In [ ]:
import os
import sys
import subprocess

def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    print("[Colab] Detected Google Colab Environment")
    
    # Clone the repository if not already present
    if not os.path.exists('goodharts_law'):
        print("[Colab] Cloning repository...")
        !git clone https://github.com/FIM43-Redeye/goodharts_law.git
    else:
        print("[Colab] Repository already cloned. Pulling latest changes...")
        %cd goodharts_law
        !git pull
        %cd ..
    
    # Install in editable mode for proper imports
    print("[Colab] Installing package in editable mode...")
    %cd goodharts_law
    %pip install -e . -q
    print("[Colab] Setup Complete")
else:
    print("[Local] Detected Local / Dev Container Environment")
    # Assume already installed via pip install -e . or running from project root

## 2. Verify Environment
Check device and package installation.

In [ ]:
import torch
from goodharts.utils.device import get_device

device = get_device()
print(f"Running on device: {device}")
if torch.cuda.is_available():
    print(f"GPU Model: {torch.cuda.get_device_name(0)}")

## 3. Run Experiment
Configure parameters and run training via CLI.

In [ ]:
# --- Configuration ---
MODE = 'ground_truth'       # Options: ground_truth, ground_truth_handhold, proxy_ill_adjusted, etc.
TIMESTEPS = 50_000          # Total training steps
N_ENVS = 128                # Parallel environments (higher = faster, more RAM)
USE_AMP = True              # Automatic Mixed Precision (faster on GPU)

# Build command
amp_flag = '--use-amp' if USE_AMP else '--no-amp'

print(f"Starting Training: {MODE} ({TIMESTEPS:,} steps)")
!python -m goodharts.training.train_ppo --mode {MODE} --timesteps {TIMESTEPS} --n-envs {N_ENVS} {amp_flag}

## 4. Analysis
Visualize the training results.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os

# Find latest log file
log_files = glob.glob(f'logs/{MODE}/*/training_log.csv')
if log_files:
    latest_log = max(log_files, key=os.path.getmtime)
    print(f"Plotting data from: {latest_log}")
    
    df = pd.read_csv(latest_log)
    
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(df['step'], df['avg_reward'], label='Reward')
    plt.xlabel('Steps')
    plt.ylabel('Average Reward')
    plt.title('Training Reward')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(df['step'], df['policy_loss'], label='Policy Loss', color='orange')
    plt.plot(df['step'], df['value_loss'], label='Value Loss', color='red', alpha=0.5)
    plt.xlabel('Steps')
    plt.title('Losses')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("No log files found. Did training complete?")